In [1]:
import os
import seaborn as sns
import matplotlib.pyplot as plt

import SALib.analyze.morris
from SALib.sample import morris

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

from tqdm import tqdm

from infrasim.optimise import *
from infrasim.utils import *

In [2]:
def process_morris_results(problem,param_values,collected_results,num_levels):
    '''Process output from Morris sensitivitys
    '''
    Si = SALib.analyze.morris.analyze(problem,
                                      np.array(param_values),
                                      np.array(collected_results),
                                      print_to_console=False,
                                      num_levels=num_levels)

    sensitivity_results = pd.DataFrame.from_dict(Si)
    return sensitivity_results


def calculate_relative_influence(sensitivity_results):
    '''Return relative influence (%) of each parameter from sensitivity
    '''
    sensitivity = sensitivity_results.copy()
    sensitivity['rel'] = abs(sensitivity['mu'])/abs(sensitivity['mu']).sum()*100
    sensitivity['mu'] = sensitivity['mu']
    sensitivity = sensitivity[['names','rel']]
    sensitivity = sensitivity.sort_values(by='names',ascending=True).reset_index(drop=True)
    sensitivity.names = sensitivity.names.str.replace('_',' ')
    sensitivity.names = sensitivity.names.str.title()
    sensitivity.names = sensitivity.names.str.replace('Res','RES')
    sensitivity.names = sensitivity.names.str.replace('Coop','COO')
    sensitivity.names = sensitivity.names.str.replace('To','to')
    sensitivity.names = sensitivity.names.str.replace('Westbank','West Bank')
    return sensitivity

In [ ]:
#File paths
nodes = '../data/nextra/spatial/network/nodes.shp'
edges = '../data/nextra/spatial/network/edges.shp'
flows = '../data/nextra/nodal_flows/processed_flows_2030.csv'

timesteps   = None
num_levels  = 10
N           = 10

# Set up problem for sensitivity analysis # name:[lower_bound,upper_bound]
params = {'coop_res_target'                 : [0,0.5],
          'jordan_to_westbank'              : [0,10**12],
          'jordan_to_israel'                : [0,10**12],
          'israel_to_westbank'              : [0,10**12],
          'israel_to_jordan'                : [0,10**12],
          'israel_to_gaza'                  : [0,10**12],
          'westbank_to_israel'              : [0,10**12],
          'westbank_to_jordan'              : [0,10**12],
          'self_sufficiency_factor'         : [0,0.9],
         }

problem = {'num_vars'    : len(params.keys()),
           'names'       : [i for i in params.keys()],
           'bounds'      : [params[i] for i in params.keys()]
          }

# create parameter values
param_values = morris.sample(problem,N=N,
                             num_levels=num_levels,
                             local_optimization=True)

# Run analysis with the specified parameter values
collected_caps_israel   = []
collected_caps_jordan   = []
collected_caps_westbank = []
collected_caps_gaza     = []

for param_set in tqdm(param_values,total=len(param_values)):

    model_run = nextra(nodes,edges,flows,
                       timesteps=timesteps,
                       energy_objective=True,
                       scenario='COO',
                       super_sink=False,
                       super_source=False,
                       # params
                       coo_res_factor=param_set[0],
                       jordan_to_westbank=param_set[1],
                       jordan_to_israel=param_set[2],
                       israel_to_westbank=param_set[3],
                       israel_to_jordan=param_set[4],
                       israel_to_gaza=param_set[5],
                       westbank_to_israel=param_set[6],
                       westbank_to_jordan=param_set[7],
                       self_sufficiency_factor=param_set[8],
                      )
    
    # build, run, and get results
    model_run.build()
    model_run.run(pprint=False)
    model_results = model_run.get_results()
    
    # get capacities
    caps = model_results.results_capacities.groupby(by=['node','territory']).max().reset_index()
    caps = caps.groupby(by='territory').sum().reset_index()
    
    # append collected capacities
    collected_caps_israel.append(caps.loc[caps.territory == 'Israel','value'].values[0])
    collected_caps_jordan.append(caps.loc[caps.territory == 'Jordan','value'].values[0])
    collected_caps_westbank.append(caps.loc[caps.territory == 'West Bank','value'].values[0])
    collected_caps_gaza.append(caps.loc[caps.territory == 'Gaza','value'].values[0])

  0%|                                                                                                | 0/100 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-03



  1%|▊                                                                                    | 1/100 [02:38<4:20:53, 158.11s/it]

  2%|█▋                                                                                   | 2/100 [05:20<4:22:18, 160.60s/it]

  3%|██▌                                                                                  | 3/100 [08:00<4:19:07, 160.28s/it]

...

 11%|█████████▏                                                                          | 11/100 [29:41<3:57:41, 160.24s/it]

 12%|██████████                                                                          | 12/100 [32:11<3:50:11, 156.95s/it]

 13%|██████████▉                                                                         | 13/100 [34:39<3:43:51, 154.38s/it]

 14%|███████████▊                                                                        | 14/100 [37:17<3:42:47, 155.43s/it]

 15%|████████████▌                                                                       | 15/100 [39:55<3:41:26, 156.31s/it]

In [ ]:
israel_results = process_morris_results(problem,param_values,collected_caps_israel,num_levels)
jordan_results = process_morris_results(problem,param_values,collected_caps_jordan,num_levels)
westbank_results = process_morris_results(problem,param_values,collected_caps_westbank,num_levels)
gaza_results = process_morris_results(problem,param_values,collected_caps_gaza,num_levels)

In [ ]:
plt.style.use('seaborn')

f,ax=plt.subplots(nrows=2,ncols=4,figsize=(15,10))

# plot boxplot
israel_caps = pd.DataFrame({'Israel' : collected_caps_israel,})
jordan_caps = pd.DataFrame({'Jordan' : collected_caps_jordan,})
westbank_caps = pd.DataFrame({'West Bank' : collected_caps_westbank,})
gaza_caps = pd.DataFrame({'Gaza' : collected_caps_gaza,})

props = dict(boxes='pink', whiskers="darkred", medians="darkred", caps="none")

israel_caps.divide(1000).boxplot(ax=ax[0,0],showfliers=False,color=props,patch_artist=True)
jordan_caps.divide(1000).boxplot(ax=ax[0,1],showfliers=False,color=props,patch_artist=True)
westbank_caps.divide(1000).boxplot(ax=ax[0,2],showfliers=False,color=props,patch_artist=True)
gaza_caps.divide(1000).boxplot(ax=ax[0,3],showfliers=False,color=props,patch_artist=True)

# plot bar chart
calculate_relative_influence(israel_results).set_index('names').plot.barh(ax=ax[1,0],legend=False)
calculate_relative_influence(jordan_results).set_index('names').plot.barh(ax=ax[1,1],legend=False)
calculate_relative_influence(westbank_results).set_index('names').plot.barh(ax=ax[1,2],legend=False)
calculate_relative_influence(gaza_results).set_index('names').plot.barh(ax=ax[1,3],legend=False)

# formatting
ax[0,0].set_title('Israel',loc='left',fontweight='bold')
ax[0,1].set_title('Jordan',loc='left',fontweight='bold')
ax[0,2].set_title('West Bank',loc='left',fontweight='bold')
ax[0,3].set_title('Gaza',loc='left',fontweight='bold')

ax[0,0].set_xticklabels([])
ax[0,1].set_xticklabels([])
ax[0,2].set_xticklabels([])
ax[0,3].set_xticklabels([])

ax[1,1].set_yticklabels([])
ax[1,2].set_yticklabels([])
ax[1,3].set_yticklabels([])

ax[1,0].set_ylabel('')
ax[1,1].set_ylabel('')
ax[1,2].set_ylabel('')
ax[1,3].set_ylabel('')

ax[1,0].set_xlim([0,100])
ax[1,1].set_xlim([0,100])
ax[1,2].set_xlim([0,100])
ax[1,3].set_xlim([0,100])

ax[0,0].set_ylabel('Total capacity [GW]')
ax[1,1].set_xlabel('Percentage influence of variables [%]')